In [2]:
import pandas as pd
import numpy as np
import os
import subprocess

# Data Extraction

In [3]:
# Set up your Kaggle API key
os.environ["KAGGLE_USERNAME"] = "leonardosantanaol"
os.environ["KAGGLE_KEY"] = "a05f1c3f7db9e29989480d2e43b076ab"

# Download the dataset using the Kaggle API
subprocess.run(["kaggle", "datasets", "download", "-d", "faviovaz/marketing-ab-testing"])

# Optionally, unzip the downloaded file
subprocess.run(["unzip", "marketing-ab-testing.zip", "-d", "marketing-ab-testing"])

/Users/leonardosantana/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Dataset URL: https://www.kaggle.com/datasets/faviovaz/marketing-ab-testing
License(s): CC0-1.0


 19%|█▉        | 1.00M/5.23M [00:00<00:00, 9.99MB/s]


Archive:  marketing-ab-testing.zip
  inflating: marketing-ab-testing/marketing_AB.csv  


100%|██████████| 5.23M/5.23M [00:00<00:00, 17.8MB/s]


CompletedProcess(args=['unzip', 'marketing-ab-testing.zip', '-d', 'marketing-ab-testing'], returncode=0)

In [4]:
df = pd.read_csv('/Users/leonardosantana/Directory/AB_/marketing-ab-testing/marketing_AB.csv')

# EDA

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588101 entries, 0 to 588100
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     588101 non-null  int64 
 1   user id        588101 non-null  int64 
 2   test group     588101 non-null  object
 3   converted      588101 non-null  bool  
 4   total ads      588101 non-null  int64 
 5   most ads day   588101 non-null  object
 6   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(4), object(2)
memory usage: 27.5+ MB


In [6]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.rename(columns={'user id': 'user_id', 'test group': 'test_group', 'converted': 'converted', 'total ads': 'total_ads', 'most ads day': 'most_ads_day', 'most ads hour': 'most_ads_hour'}, inplace=True)
df['test_group'] = df['test_group'].map({'ad': 1, 'psa': 0})

In [7]:
df.head()

,user_id,test_group,converted,total_ads,most_ads_day,most_ads_hour
0,1069124,1,False,130,Monday,20
1,1119715,1,False,93,Tuesday,22
2,1144181,1,False,21,Tuesday,18
3,1435133,1,False,355,Tuesday,10
4,1015700,1,False,276,Friday,14


In [8]:
df.user_id.nunique() == df.shape[0]


True

In [9]:
df.describe([0.01, 0.05, 0.10, 0.20, 0.80, 0.90, 0.95, 0.99])[["total_ads"]].T


,count,mean,std,min,1%,5%,10%,20%,50%,80%,90%,95%,99%,max
total_ads,588101.0,24.820876,43.715181,1.0,1.0,1.0,2.0,3.0,13.0,33.0,57.0,88.0,202.0,2065.0


In [10]:
df.groupby("test_group").total_ads.agg(["count", "median", "mean", "std", "max"])

,count,median,mean,std,max
test_group,,,,,
0,23524,12.0,24.761138,42.860720,907
1,564577,13.0,24.823365,43.750456,2065


In [11]:
df_ad = df[df['test_group'] == 1]
df_psa = df[df['test_group'] == 0]

In [12]:
def remove_outliers(df, column):

    shape_before = df.shape

    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_clean = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    outliers_indices = df.index.difference(df_clean.index)
    outliers_count = len(outliers_indices)
    outliers_percentage = outliers_count / len(df) * 100

    shape_after = df_clean.shape

    print("The form before the removal of outliers:", shape_before)
    print("The form after the removal of outliers:", shape_after)
    print("Number of outliers:", outliers_count)
    print("Percent outliers:", outliers_percentage)

    return df_clean

In [13]:
df_ad = remove_outliers(df_ad, 'total_ads')
df_psa = remove_outliers(df_psa, 'total_ads')


The form before the removal of outliers: (564577, 6)
The form after the removal of outliers: (514716, 6)
Number of outliers: 49861
Percent outliers: 8.831567704670931
The form before the removal of outliers: (23524, 6)
The form after the removal of outliers: (21203, 6)
Number of outliers: 2321
Percent outliers: 9.86651929943887


In [14]:
df = pd.concat([df_ad, df_psa])
df

,user_id,test_group,converted,total_ads,most_ads_day,most_ads_hour
2,1144181,1,False,21,Tuesday,18
7,1496843,1,False,17,Sunday,18
8,1448851,1,False,21,Tuesday,19
11,1637531,1,False,47,Wednesday,13
12,1081965,1,False,61,Tuesday,20
...,...,...,...,...,...,...
588052,900959,0,False,16,Tuesday,22
588063,902828,0,False,3,Tuesday,22
588066,914578,0,False,1,Tuesday,22
588069,909042,0,False,6,Tuesday,22


In [21]:
df.to_csv('AB_testing.csv', index=False)